In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
RANDOM_STATE = 13

## Load Data

In [3]:
data_df = pd.read_csv("../data/ml-latest-small/ratings.csv")

In [4]:
data_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
data_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
userId       100836 non-null int64
movieId      100836 non-null int64
rating       100836 non-null float64
timestamp    100836 non-null int64
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


## Split Training And Test Set

In [7]:
test_df = data_df.sample(frac = 0.2, random_state=RANDOM_STATE)
train_df = data_df.drop(test_df.index)
print(train_df.shape, test_df.shape)

(80669, 4) (20167, 4)


In [8]:
test_df.head(5)

,userId,movieId,rating,timestamp
48260,313,293,4.0,1030555572
80555,509,6888,2.5,1435994604
51069,330,1387,4.0,1285905302
83115,526,308,3.5,1502132664
49813,318,134214,4.0,1451995959


## Matrix Factorization
### Algorithm and Regularization
Here I perform stochastic gradien descent to do matrix factorization. Your algorithm should consider case that there are new users and movies adding to the dataset you used to train. In other words, the dimension your matrix R, q, p is dynamic.

* For algorithms, the referenced paper are:

    * A1. **Stochastic Gradient Descent Section(P1%20Recommender-Systems.pdf)**: Learning Algorithms-Stochastic Gradient Descent

    * A2. Gradient Descent with Probabilistic Assumptions Section 2

    * A3. Alternating Least Squares Section 3.1
        * For regularizations, the referenced paper are:

* Regularization
    * R1. Penalty of Magnitudes Section: a Basic Matrix Factorization Model

    * R2. Bias and Intercepts Section: Adding Biases

    * R3. Temporal Dynamics Section 4
    
> Our Group(Sec1 Group4) only need to implement
> 1. A1+R3+P2
> 2. A1+R3+P3

In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def single_error(target_row, predicted_df):
    """
    Calculate the squared error for a specifc rating, compared with the prediction
    This function is used by 'apply' method, don't call it yourself
    
    """
    userId = int(target_row['userId'])
    movieId = int(target_row['movieId'])
    return (target_row['rating'] - predicted_df.loc[userId, movieId]) **2


def stochasticGD(data, train, test, max_iter, f_dim = 10, learning_rate = 0.001, 
                 regularization = 0.3, stopping_threshold = 0.001, random_state = RANDOM_STATE):
    """
    :param data: the overall data, which is needed to record all movie id and user id
    :param train: training dataset in pd.DataFrame
    :param test: test dataset in pd.DataFrame
    :param max_ter: number of gradient descent iteration
    :param f_dim: dimension for the SVD decomposition
    :param learning_rate: gradient descent learning rate
    :param regularization: regularization factor
    :param stopping_threshold: the upper bound to not update vector. If the calculated gradient is larger than this,
        the corresponding vector will not be updated.
    
    :return:
    """
    # set random state
    users = data['userId'].unique()
    movies = data['movieId'].unique()
    
    # create factorized matrix
    np.random.seed(random_state)
    # intialize to rand(-1,1)
    users_df = pd.DataFrame(np.random.rand(len(users), f_dim) * 2 - 1, index=users)
    movies_df = pd.DataFrame(np.random.rand(len(movies),f_dim) * 2 - 1, index=movies)
    
    # record training RMSE
    train_rmse = []
    test_rmse = []
    
    # stochastic gradient descent
    for iteration in range(max_iter):
        # shuffle the index for this stochastic iteration
        sample_indices = np.array(train_df.index)
        np.random.shuffle(sample_indices)
        
        for sample_i in sample_indices:
            sample = train_df.loc[sample_i,:]
            # calculate loss
            r_ui = sample['rating']
            user = users_df.loc[int(sample['userId']), :].copy()
            movie = movies_df.loc[int(sample['movieId']), :].copy()
            e_ui = r_ui - np.dot( user, movie)    
            # update vector
            gradient_movie = e_ui * user - regularization * movie
            if np.any(np.abs(gradient_movie) > stopping_threshold):
                movies_df.loc[int(sample['movieId']), :] = movie + learning_rate * gradient_movie

            gradient_user = e_ui * movie - regularization  * user
            if np.any(np.abs(gradient_user) > stopping_threshold):
                users_df.loc[int(sample['userId']), :] = user + learning_rate * gradient_user
        
        # make prediction. shape is (users * movies)
        predicted_df = pd.DataFrame(np.dot(users_df, movies_df.T), columns= movies_df.index, index = users_df.index)

        # calculating rmse
        if iteration % 10 == 0:
            train_single_rmse = np.sqrt(np.mean(train.apply(single_error, axis = 1, predicted_df = predicted_df)))
            train_rmse.append(train_single_rmse)
            test_single_rmse = np.sqrt(np.mean(test.apply(single_error, axis = 1, predicted_df = predicted_df)))
            test_rmse.append(test_single_rmse)
            print(f"Epoch: {iteration}")
            print(f"Training RMSE: {train_single_rmse}\t Test RMSE: {test_single_rmse}",)
            
    return users_df, movies_df, train_rmse, test_rmse

In [10]:
users_df, movies_df, train_rmse, test_rmse = stochasticGD(data_df, train_df, test_df, 1)

Epoch: 0
Training RMSE: 3.7232815485207023	 Test RMSE: 3.741565397631318


In [11]:
predicted_df = pd.DataFrame(np.dot(users_df, movies_df.T), columns= movies_df.index, index = users_df.index)
predicted_df.head(5)

,1,3,6,47,50,70,101,110,151,157,...,147662,148166,149011,152372,158721,160341,160527,160836,163937,163981
1,0.420647,0.666458,-0.572329,-0.042071,1.906138,-0.078009,-0.821778,0.009163,0.991631,-1.415281,...,0.308248,-0.630935,-0.480102,-0.530810,0.170600,0.591710,0.668683,-0.499309,1.045802,-1.666364
2,-1.729266,-1.348801,-0.545158,0.065139,-1.013823,0.291876,1.967949,0.256932,0.872188,0.668263,...,0.433660,0.863355,0.924489,0.157198,-1.705613,-1.557026,0.896567,0.002667,-0.349469,0.563209
3,0.436003,-0.308128,-1.622448,0.402658,0.223470,2.055105,1.151883,-1.357072,-0.125645,-1.424658,...,1.716859,-0.362625,0.150071,-1.955613,1.567590,0.824677,0.114928,-0.647488,3.061716,-0.431924
4,0.613158,0.426844,1.082139,0.422054,0.291269,-0.197889,-0.220295,-0.452328,-0.851370,1.195279,...,-0.428659,1.655227,0.803953,-0.991743,-0.393152,0.720676,0.087631,-1.385409,-0.944979,0.282721
5,-0.387320,-0.603773,0.037303,-0.360848,0.412845,-1.841770,0.122034,0.912422,-0.426932,-0.563312,...,-0.522372,-1.220036,0.251329,0.826388,0.566947,-0.454132,-0.331613,0.131828,0.057954,0.318165


# Regularization

In [3]:
# calling data
ratings = pd.read_csv("../data/ml-latest-small/ratings.csv")
movies = pd.read_csv("../data/ml-latest-small/movies.csv")
tags = pd.read_csv("../data/ml-latest-small/tags.csv")
links = pd.read_csv("../data/ml-latest-small/links.csv")

In [9]:
ratings.head()
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
test = pd.merge(ratings, movies)
test.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [30]:
# Basic Equation
# rui = mu + bi + bu + qi^T * pu
mu = ratings['rating'].mean()
mu

##bi
avg_rating_by_movie = ratings.groupby('movieId').mean() #average rating by each users
overall_movie_rating = avg_rating_by_movie['rating'].mean()
# if movieId = 10
each_movie_rating = ratings.loc[ratings['movieId'] == 10, ['rating']].mean() #average rating for specific movie
bi = each_movie_rating - overall_movie_rating #deviance from average

##bu
avg_rating_by_user = ratings.groupby('userId').mean() #average rating by each users
overall_user_rating = avg_rating_by_user['rating'].mean()
# if userId = 10
each_user_rating = ratings.loc[ratings['userId'] == 10, ['rating']].mean() #average rating for specific user
bu = each_user_rating - overall_user_rating #deviance from average

## dont know about qi^T, pu(t)
rui = mu + bi + bu

## Implementing time

In [147]:
# Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
# rui = mu + bi(t) + bu(t) + qi^T * pu(t)

binsize = 50 #50 years of data
#use distinct item bias based on bins
#100836/50 = 2016.72 rows per bin, so just use 2017 per bins
time_ratings = ratings.sort_values(by='timestamp', ascending=False) # sorting data by timestamp
time_ratings = pd.DataFrame(time_ratings)
bins = [0] * 50 # list of 50 bins
bins[0] = ratings.loc[0:2016,]
for i in range(1,48):
    bins[i] = ratings.loc[i*2017:2016+2017*i,]
bins[49] = ratings.loc[98833:,]

#average ratings for each bins
avg_bin_ratings = [0] * 50
#for i in range(0,49):
#    avg_bin_ratings[i] = bins[i]['rating'].mean()

#bit = bi 


In [154]:
bins[0]['rating'].mean()

3.6544372830937033

In [159]:
avg_bin_ratings = [0] * 50
for j in range(0,49):
    avg_bin_ratings[j] = bins[j]['rating'].mean()
avg_bin_ratings

TypeError: 'int' object is not subscriptable

In [129]:
# normalize timestamp data?
ratings.shape
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [130]:
time_ratings.shape
time_ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [61]:
ratings['timestamp'].max() - ratings['timestamp'].min()

709674635

In [64]:
ratings['timestamp'].count() #100836 rows for 50 years

100836

In [65]:
100836/50

2016.72

range(0, 50)
